In [1]:
import os
import time
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
import autokeras as ak
import pandas as pd
import numpy as np

In [ ]:
while True:
    nbr = len(os.listdir('data/train'))
    print(nbr)
    if nbr >= 2295:
        break
    time.sleep(5)

In [5]:
def load_images(directory_path, size):
    images = []
    for filename in tqdm(os.listdir(directory_path)):
        if filename.endswith('.jpg'):
            img = image.load_img(os.path.join(directory_path, filename), target_size=(size, size))
            images.append(image.img_to_array(img))
    return np.array(images)

In [6]:
x_train_raw = load_images('data/train', 224)

100%|██████████| 2296/2296 [00:53<00:00, 43.17it/s]


In [7]:
model = ak.ImageClassifier(max_trials=1)

In [8]:
y_train_raw = pd.read_csv('data/train_labels.csv')

In [11]:
y_train = y_train_raw['invasive'].to_numpy()

In [13]:
model.fit(x_train_raw, y_train, epochs=1)

Trial 1 Complete [00h 02m 02s]
val_loss: 0.6936550736427307

Best val_loss So Far: 0.6936550736427307
Total elapsed time: 00h 02m 02s
INFO:tensorflow:Oracle triggered exit
72/72 [==============================] - 121s 2s/step - loss: 3.0015 - accuracy: 0.5085
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


In [14]:
x_test = load_images('data/test', 224)

100%|██████████| 1531/1531 [00:33<00:00, 46.33it/s]


In [18]:
y_pred = model.predict(x_test)

In [21]:
y_pred = y_pred.reshape(1531)

In [24]:
def get_labels():
    labels = []
    for filename in os.listdir('data/test'):
        label, _ = os.path.splitext(filename)
        labels.append(label)
    return labels

In [25]:
df_pred = pd.DataFrame(
  {
    'name': get_labels(),
    'invasive': y_pred,
  }
)

In [28]:
df_pred.to_csv('data/prediction.csv', index=False)